In [1]:
import nltk
import numpy as np
import tensorflow as tf
from tensorflow.python.client import timeline
import networkx as nx
from collections import defaultdict,namedtuple,Counter
from glob import glob
import sys
import os
import math
import random
from six.moves import xrange
if sys.version_info[0] >= 3:
    unicode = str

import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,train_test_split
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec

random.seed(0)
np.random.seed(0)



C:\Users\my\Miniconda3\envs\ml_python\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-11-14 00:06:31,299 'pattern' package not found; tag filters are not available for English


In [2]:
CORA = namedtuple('CORA', 'words tags')

datasets = []
labels = defaultdict(list)
with open("cora_content.txt") as f:
    for line in f:
        line = line.split()
        ID = line[0]
        labels[line[-1]].append(ID)
        words = []
        for i,w in enumerate(line[1:-1]):
            if w == "1":
                words.append(str(i))
        datasets.append(
            CORA(
                words,
                [ID]
            )
        )

logging.info("done... %s papers loaded" % (len(datasets)))
logging.info("%s labels" % (len(labels)))

datasets[0:8]

2017-11-14 00:06:31,770 done... 2708 papers loaded
2017-11-14 00:06:31,770 7 labels


[CORA(words=['118', '125', '176', '252', '351', '456', '507', '521', '619', '648', '698', '702', '734', '845', '902', '1205', '1209', '1236', '1352', '1426'], tags=['31336']),
 CORA(words=['12', '509', '620', '763', '882', '893', '978', '1131', '1135', '1177', '1207', '1256', '1263', '1266', '1332', '1389', '1425'], tags=['1061127']),
 CORA(words=['45', '209', '212', '239', '292', '394', '510', '514', '581', '621', '623', '638', '1075', '1132', '1177', '1206', '1263', '1289', '1349', '1389', '1415', '1421'], tags=['1106406']),
 CORA(words=['41', '93', '99', '149', '594', '617', '624', '648', '874', '915', '942', '988', '1004', '1049', '1071', '1170', '1177', '1194', '1292', '1348', '1349'], tags=['13195']),
 CORA(words=['44', '122', '135', '153', '364', '396', '402', '474', '507', '619', '661', '699', '701', '828', '1066', '1174', '1175', '1177', '1208', '1209', '1212', '1254', '1381'], tags=['37879']),
 CORA(words=['93', '168', '211', '507', '526', '551', '874', '972', '1143', '1177',

In [3]:
model = Doc2Vec(alpha=0.025, window=10, min_count=10, min_alpha=0.025, size=100)
model.build_vocab(datasets)

# decrease alpha
for i in range(100):
    random.shuffle(datasets)
    model.alpha = 0.025-0.002*i
    model.min_alpha = model.alpha
    model.train(datasets,total_examples=model.corpus_count,epochs=model.iter)


2017-11-14 00:06:31,787 collecting all words and their counts
2017-11-14 00:06:31,790 PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-11-14 00:06:31,810 collected 1432 word types and 2708 unique tags from a corpus of 2708 examples and 49216 words
2017-11-14 00:06:31,811 Loading a fresh vocabulary
2017-11-14 00:06:31,813 min_count=10 retains 968 unique words (67% of original 1432, drops 464)
2017-11-14 00:06:31,817 min_count=10 leaves 46251 word corpus (93% of original 49216, drops 2965)
2017-11-14 00:06:31,822 deleting the raw counts dictionary of 1432 items
2017-11-14 00:06:31,824 sample=0.001 downsamples 76 most-common words
2017-11-14 00:06:31,825 downsampling leaves estimated 39952 word corpus (86.4% of prior 46251)
2017-11-14 00:06:31,827 estimated required memory for 968 words and 100 dimensions: 2883200 bytes
2017-11-14 00:06:31,831 resetting layer weights
2017-11-14 00:06:31,883 training model with 3 workers on 968 vocabulary and 100 features, using 

2017-11-14 00:06:36,258 training model with 3 workers on 968 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-11-14 00:06:36,603 worker thread finished; awaiting finish of 2 more threads
2017-11-14 00:06:36,606 worker thread finished; awaiting finish of 1 more threads
2017-11-14 00:06:36,608 worker thread finished; awaiting finish of 0 more threads
2017-11-14 00:06:36,609 training on 246080 raw words (213239 effective words) took 0.3s, 620016 effective words/s
2017-11-14 00:06:36,610 under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-11-14 00:06:36,613 training model with 3 workers on 968 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-11-14 00:06:36,955 worker thread finished; awaiting finish of 2 more threads
2017-11-14 00:06:36,962 worker thread finished; awaiting finish of 1 more threads
2017-11-14 00:06:36,968 worker thread finished; awaiting finish of 0 more thread

2017-11-14 00:06:41,433 worker thread finished; awaiting finish of 1 more threads
2017-11-14 00:06:41,435 worker thread finished; awaiting finish of 0 more threads
2017-11-14 00:06:41,436 training on 246080 raw words (213317 effective words) took 0.4s, 501783 effective words/s
2017-11-14 00:06:41,437 under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-11-14 00:06:41,440 training model with 3 workers on 968 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-11-14 00:06:41,827 worker thread finished; awaiting finish of 2 more threads
2017-11-14 00:06:41,834 worker thread finished; awaiting finish of 1 more threads
2017-11-14 00:06:41,835 worker thread finished; awaiting finish of 0 more threads
2017-11-14 00:06:41,836 training on 246080 raw words (213385 effective words) took 0.4s, 550770 effective words/s
2017-11-14 00:06:41,837 under 10 jobs per worker: consider setting a smaller `batch_words' for smoother al

2017-11-14 00:06:46,299 under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-11-14 00:06:46,302 training model with 3 workers on 968 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-11-14 00:06:46,648 worker thread finished; awaiting finish of 2 more threads
2017-11-14 00:06:46,651 worker thread finished; awaiting finish of 1 more threads
2017-11-14 00:06:46,662 worker thread finished; awaiting finish of 0 more threads
2017-11-14 00:06:46,665 training on 246080 raw words (213168 effective words) took 0.4s, 597106 effective words/s
2017-11-14 00:06:46,666 under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-11-14 00:06:46,670 training model with 3 workers on 968 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-11-14 00:06:47,022 worker thread finished; awaiting finish of 2 more threads
2017-11-14 00:06:47,028 worker thread finished

2017-11-14 00:06:51,458 worker thread finished; awaiting finish of 2 more threads
2017-11-14 00:06:51,466 worker thread finished; awaiting finish of 1 more threads
2017-11-14 00:06:51,469 worker thread finished; awaiting finish of 0 more threads
2017-11-14 00:06:51,470 training on 246080 raw words (213417 effective words) took 0.3s, 624092 effective words/s
2017-11-14 00:06:51,471 under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-11-14 00:06:51,474 training model with 3 workers on 968 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-11-14 00:06:51,811 worker thread finished; awaiting finish of 2 more threads
2017-11-14 00:06:51,820 worker thread finished; awaiting finish of 1 more threads
2017-11-14 00:06:51,824 worker thread finished; awaiting finish of 0 more threads
2017-11-14 00:06:51,825 training on 246080 raw words (213127 effective words) took 0.3s, 616575 effective words/s
2017-11-14 00:06:51,826 

2017-11-14 00:06:56,062 training on 246080 raw words (213367 effective words) took 0.4s, 600380 effective words/s
2017-11-14 00:06:56,063 under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-11-14 00:06:56,065 training model with 3 workers on 968 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-11-14 00:06:56,403 worker thread finished; awaiting finish of 2 more threads
2017-11-14 00:06:56,407 worker thread finished; awaiting finish of 1 more threads
2017-11-14 00:06:56,413 worker thread finished; awaiting finish of 0 more threads
2017-11-14 00:06:56,413 training on 246080 raw words (213312 effective words) took 0.3s, 622920 effective words/s
2017-11-14 00:06:56,414 under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-11-14 00:06:56,417 training model with 3 workers on 968 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-11-14 00:

2017-11-14 00:07:00,713 training model with 3 workers on 968 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-11-14 00:07:01,055 worker thread finished; awaiting finish of 2 more threads
2017-11-14 00:07:01,060 worker thread finished; awaiting finish of 1 more threads
2017-11-14 00:07:01,065 worker thread finished; awaiting finish of 0 more threads
2017-11-14 00:07:01,066 training on 246080 raw words (213311 effective words) took 0.3s, 615258 effective words/s
2017-11-14 00:07:01,066 under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-11-14 00:07:01,069 training model with 3 workers on 968 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-11-14 00:07:01,410 worker thread finished; awaiting finish of 2 more threads
2017-11-14 00:07:01,420 worker thread finished; awaiting finish of 1 more threads
2017-11-14 00:07:01,422 worker thread finished; awaiting finish of 0 more thread

2017-11-14 00:07:05,854 worker thread finished; awaiting finish of 1 more threads
2017-11-14 00:07:05,861 worker thread finished; awaiting finish of 0 more threads
2017-11-14 00:07:05,862 training on 246080 raw words (213328 effective words) took 0.4s, 550936 effective words/s
2017-11-14 00:07:05,863 under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay
2017-11-14 00:07:05,866 training model with 3 workers on 968 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2017-11-14 00:07:06,205 worker thread finished; awaiting finish of 2 more threads
2017-11-14 00:07:06,213 worker thread finished; awaiting finish of 1 more threads
2017-11-14 00:07:06,218 worker thread finished; awaiting finish of 0 more threads
2017-11-14 00:07:06,218 training on 246080 raw words (213294 effective words) took 0.3s, 615720 effective words/s
2017-11-14 00:07:06,219 under 10 jobs per worker: consider setting a smaller `batch_words' for smoother al

In [4]:
X = []
Y = []
with open('doc2vec.embd','w') as f:
    f.write("%s %s\n"%(len(datasets),100))
    for y,key in enumerate(labels.keys()):
        for index,paper in enumerate(labels[key]):
            f.write(paper+" "+" ".join([str(x) for x in model.docvecs[paper]])+"\n")
            X.append(model.docvecs[paper])
            Y.append(y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=0)
clf = SVC(kernel='rbf',C=1.5).fit(X_train,y_train)
print(clf.score(X_test, y_test))

parameters = {
    "kernel":["rbf"],
    "C" :[1.5]
             }
tunedclf = GridSearchCV(clf,parameters,cv=10,n_jobs=24)
tunedclf.fit(X,Y)
print("scores %s" % tunedclf.best_score_)

0.501477104874
scores 0.562038404727


In [13]:
G = defaultdict(dict)

for data in datasets:
    for n in model.docvecs.most_similar(data.tags,topn=2):
        G[data.tags[0]][n[0]] = None
        G[n[0]][data.tags[0]] = None

with open('cora_cites.txt') as f:
    for line in f:
        line = line.rstrip().split("\t")
        try:
            G[line[0]][line[1]] = None
            G[line[1]][line[0]] = None
        except:
            print(line)
            
neighbors = []

for i in range(10):
    for node in G:
        path = [node]
        while len(path) < 60:
            cur = path[-1]
            path.append(random.choice(list(G[cur].keys())))
        neighbors.append(path)
        
        
print(len(neighbors))
print(len(neighbors[0]))


27080
60


In [6]:
p2v = Word2Vec(size=100, window=5, min_count=0)
p2v.build_vocab(neighbors)
p2v.intersect_word2vec_format('doc2vec.embd',lockf=1.0)
p2v.train(neighbors,total_examples=p2v.corpus_count,epochs=p2v.iter)


2017-11-12 12:03:08,204 collecting all words and their counts
2017-11-12 12:03:08,206 PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-11-12 12:03:08,249 PROGRESS: at sentence #10000, processed 400000 words, keeping 2708 word types
2017-11-12 12:03:08,298 PROGRESS: at sentence #20000, processed 800000 words, keeping 2708 word types
2017-11-12 12:03:08,330 collected 2708 word types from a corpus of 1083200 raw words and 27080 sentences
2017-11-12 12:03:08,331 Loading a fresh vocabulary
2017-11-12 12:03:08,339 min_count=0 retains 2708 unique words (100% of original 2708, drops 0)
2017-11-12 12:03:08,340 min_count=0 leaves 1083200 word corpus (100% of original 1083200, drops 0)
2017-11-12 12:03:08,346 deleting the raw counts dictionary of 2708 items
2017-11-12 12:03:08,347 sample=0.001 downsamples 4 most-common words
2017-11-12 12:03:08,348 downsampling leaves estimated 1075207 word corpus (99.3% of prior 1083200)
2017-11-12 12:03:08,348 estimated required memory for

5376121

In [12]:
X = []
Y = []
for y,key in enumerate(labels.keys()):
    for index,paper in enumerate(labels[key]):
        X.append(p2v[paper])
        Y.append(y)
print(X[0])
print(Y)

[-0.71758455 -0.66209936  0.55502558 -0.6359151   1.1250695  -1.21198142
  0.23451109 -1.23037171 -0.1503356   0.73392546 -2.06334996 -0.36316052
  0.49197263  1.66867948  1.02787614  1.40376604  0.77955592  0.98882478
  0.47122076 -0.2451476   1.11152911  1.89864004  0.23590325  0.02930315
  0.11828229 -1.43437624  1.1972779  -0.54909819  0.91523319  1.17575431
  1.18605649 -0.88767278  1.67310548 -0.43302917  1.68476307 -0.60187632
 -0.6068747  -1.53292167  1.60054517  1.87925768 -1.18226838  1.35247755
 -1.19235623 -1.9220295   0.06113619 -0.78426415 -1.46965313 -0.82058495
  0.43600863 -1.20224631 -0.1054199  -0.28657159 -0.2002437  -0.28128338
  0.94277942 -0.12257486  0.30342183 -0.01108181  1.00861275  0.0807043
  0.16269824  0.77411056  1.50581765  0.03296845  1.76792169  2.53235221
 -0.08404452 -1.31786275  0.62807137  0.2615113   1.11879659  0.64269465
  0.50509471 -1.10541523  1.84277332 -2.20028782  0.09468788 -0.6896438
 -2.08605242 -2.37635541 -1.29334199  0.45332319  1.0

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=42)
clf = SVC(kernel='rbf',C=1.5).fit(X_train,y_train)
print(clf.score(X_test, y_test))
clf = SVC()
parameters = {
    "kernel":["rbf"],
    "C" :[1,10,100]
             }
tunedclf = GridSearchCV(clf,parameters,cv=10,n_jobs=24)
tunedclf.fit(X,Y)
logging.info("scores %s" % tunedclf.best_score_)

0.801329394387


2017-11-12 12:03:24,732 scores 0.819423929099
